# Bot-vs-bot reverts
## Getting and processing data

In [1]:
# bot-vs-bot revert table: https://quarry.wmflabs.org/query/17237

!wget https://quarry.wmflabs.org/run/161195/output/0/tsv?download=true -O enwiki-botvbot-pagetitles.tsv

--2017-03-10 21:44:24--  https://quarry.wmflabs.org/run/161195/output/0/tsv?download=true
Resolving quarry.wmflabs.org (quarry.wmflabs.org)... 10.68.21.68
Connecting to quarry.wmflabs.org (quarry.wmflabs.org)|10.68.21.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘enwiki-botvbot-pagetitles.tsv’

enwiki-botvbot-page     [               <=>    ]  36.78M  3.61MB/s   in 11s    

2017-03-10 21:44:35 (3.41 MB/s) - ‘enwiki-botvbot-pagetitles.tsv’ saved [38564767]



In [19]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

!pip install ftfy
import ftfy

  Running setup.py install for ftfy ... - \ done


In [3]:
df = pd.read_csv("enwiki-botvbot-pagetitles.tsv", sep="\t", encoding='utf-8')
len(df)

158606

In [4]:
print(df.columns)

Index(['rev_id', 'rev_timestamp', 'rev_user', 'rev_user_text', 'rev_page',
       'rev_sha1', 'rev_minor_edit', 'rev_deleted', 'rev_parent_id',
       'archived', 'reverting_id', 'reverting_timestamp', 'reverting_user',
       'reverting_user_text', 'reverting_page', 'reverting_sha1',
       'reverting_minor_edit', 'reverting_deleted', 'reverting_parent_id',
       'reverting_archived', 'rev_revert_offset', 'revisions_reverted',
       'reverted_to_rev_id', 'page_namespace', 'page_title'],
      dtype='object')


In [5]:
df[0:5]

,rev_id,rev_timestamp,rev_user,rev_user_text,rev_page,rev_sha1,rev_minor_edit,rev_deleted,rev_parent_id,archived,...,reverting_sha1,reverting_minor_edit,reverting_deleted,reverting_parent_id,reverting_archived,rev_revert_offset,revisions_reverted,reverted_to_rev_id,page_namespace,page_title
0,56161718,20060531172522,91310,CanisRufus,584516,1lyohbi8ymubjfzdb4w4ssfiflfpgat,1,0,54170358,0,...,b3lf3olmh1hw99f68tjh187jl7dzi1j,1,0,56161718,0,1,1,54170358,0,Sharon_Pratt_Kelly
1,56161633,20060531172452,91310,CanisRufus,793703,bxfv9z3d7uypsla28qfioc5qs4l963w,1,0,54169562,0,...,1nlk1pfhntoa3m0inrdpotrqb5ajxif,1,0,56161633,0,1,1,54169562,0,Anthony_A._Williams
2,103117636,20070125105128,234358,Mathbot,5202035,6ruzx0gegy5xy0mw5yxsdnh1fir7mwd,0,0,102641383,0,...,c6uu9wkg34ko0nidqakdktfkkvcds3k,0,0,104315087,0,1,3,102641383,4,Version_1.0_Editorial_Team/Core_topics_article...
3,105582355,20070204154028,234358,Mathbot,6744678,p14vq3lzxeu50upo1ax9h3ya2hrz1hv,0,0,104618440,0,...,qaiydx752b8av3qp2zbnfpqspttpw91,0,0,105582355,0,1,1,104618440,4,Version_1.0_Editorial_Team/Medicine_articles_b...
4,105627219,20070204194135,234358,Mathbot,8468041,rebsad1ch2hvs85nhf0nwm2r8ukboil,0,0,104982162,0,...,3lda2892o6z3p29r8xipbqh780h2rph,0,0,105627219,0,1,1,104982162,4,Version_1.0_Editorial_Team/Soft_drinks_article...


In [6]:
df['reverting_timestamp_dt'] = pd.to_datetime(df['reverting_timestamp'], format="%Y%m%d%H%M%S")

In [7]:
df = df.set_index('reverting_timestamp_dt')

In [8]:
#df = df[df['page_namespace'] == 0].copy()

In [21]:
df['usertext_pair'] = df['reverting_user_text'] + " reverting " + df['rev_user_text']
df['usertext_pair'].value_counts().head()

AttributeError: 'Series' object has no attribute 'find'

In [10]:
df['usertext_pair'].values

array(['Cydebot reverting CanisRufus', 'Cydebot reverting CanisRufus',
       'WP 1.0 bot reverting Mathbot', ...,
       'Citation bot reverting Citation bot 2',
       'AvicBot reverting EmausBot', 'DumbBOT reverting Lowercase sigmabot'], dtype=object)

In [11]:
gp = df.groupby(['usertext_pair'])

In [12]:
df['page_namespace'].value_counts()

0      83295
4      31227
14     27417
1      10354
3       3365
2       1350
10       890
5        253
11       204
6        177
109       28
100       27
7          5
108        4
101        3
118        3
12         2
15         2
Name: page_namespace, dtype: int64

In [13]:
def botpair_ns_table(botpair_df):
    tablestr = "Reverts by namespace<br>\n"
    for x, y in botpair_df.page_namespace.value_counts().iteritems():
        namespaces_dict={0: "Article:", 
                         1: "Talk:",
                         2:"User:",
                         3: "User_talk:",
                         4:"Wikipedia:",
                         5: "Wikipedia_talk:",
                         6:"File:",
                         7:"File_talk:",
                         8:"MediaWiki:",
                         9:"MediaWiki_talk:",
                         10:"Template:", 
                         11:"Template_talk:",
                         12:"Help:",
                         13:"Help_talk:",
                         14:"Category:", 
                         15:"Category_talk:",
                         100:"Portal:",
                         101:"Portal_talk:",
                         108:"Book:",
                         109:"Book_talk:",
                         118:"Draft:"}
    
        tablestr = tablestr + namespaces_dict[x] + " " + str(y) + "<br>\n"
    return tablestr
    

In [14]:
namespaces_dict={0: "", 
                 1: "Talk:",
                 2:"User:",
                 3: "User_talk:",
                 4:"Wikipedia:",
                 5: "Wikipedia_talk:",
                 6:"File:",
                 7:"File_talk:",
                 8:"MediaWiki:",
                 9:"MediaWiki_talk:",
                 10:"Template:", 
                 11:"Template_talk:",
                 12:"Help:",
                 13:"Help_talk:",
                 14:"Category:", 
                 15:"Category_talk:",
                 100:"Portal:",
                 101:"Portal_talk:",
                 108:"Book:",
                 109:"Book_talk:",
                 118:"Draft:"}

In [20]:
with open("botvbot.html", "w") as f:
    f.write("<h1>Bot vs bot reverts on enwiki</h1>")
    f.write("<p>For bot-bot pairs with more than 20 reverts</p>")
    f.write("<p>Jupyter notebook for this code <a href=\"http://paws-public.wmflabs.org/paws-public/User:Staeiou/wiki-stat-notebooks/botvbot/exploratory-botplots-samples.ipynb#\"> here</a></p>")
    for botpair in df['usertext_pair'].unique():
        botpair_df = df[df['usertext_pair'] == botpair]
        total_reverts = len(botpair_df)
        if total_reverts > 20:
            
            first_revert = df[df['usertext_pair'] == botpair].index.min()
            last_revert = df[df['usertext_pair'] == botpair].index.max()
            sample = df[df['usertext_pair'] == botpair].sample(2)    
            count = 0
            f.write("<h2>" + ftfy.fix_text(botpair) + "</h2>\n")
            f.write("<p>Total reverts: " + str(total_reverts) + "</p>\n")
            f.write(botpair_ns_table(botpair_df))
            f.write("<p>First revert: " + str(first_revert) + "</p>\n")
            f.write("<p>Last revert: " + str(last_revert) + "</p>\n")
            f.write("<p>Random examples: ")
                        
            for revid in df[df['usertext_pair'] == botpair].sample(20).iterrows():
                page_title = namespaces_dict[revid[1].page_namespace] + revid[1].page_title
                
                f.write("<a href=\"https://en.wikipedia.org/w/index.php?title=" + page_title + "&diff=prev&oldid=" + str(revid[1].reverting_id) + "\">" + str(count) + "</a> ")
                count = count + 1

In [49]:
output_df = pd.DataFrame(columns=["reverting_bot", "reverted_bot", "total_reverts", "ns0_reverts", "ex0", "ex1", "ex2", "ex3", "ex4", "ex5", "ex6", "ex7", "ex8", "ex9", "ex10"])

with open("botvbot.tsv", "w") as f:
    for botpair in df['usertext_pair'].unique():
        botpair_df = df[df['usertext_pair'] == botpair]
        total_reverts = len(botpair_df)
        ns0_reverts = len(botpair_df[botpair_df['page_namespace'] == 0])
        if total_reverts > 20:
            sample = df[df['usertext_pair'] == botpair].sample(1)  
            
            reverting_user_text_link = "=HYPERLINK(\"http://enwp.org/User:" + sample.reverting_user_text[0] + "\",\"" + sample.reverting_user_text[0] + "\")"
            rev_user_text_link = "=HYPERLINK(\"http://enwp.org/User:" + sample.rev_user_text[0] + "\",\"" + sample.rev_user_text[0] + "\")"

            first_revert = df[df['usertext_pair'] == botpair].index.min()
            last_revert = df[df['usertext_pair'] == botpair].index.max()

            ex = {}
            count = 0

            for revid in df[df['usertext_pair'] == botpair].sample(10).iterrows():
                page_title = namespaces_dict[revid[1].page_namespace] + revid[1].page_title
                ex[count] = "=HYPERLINK(\"https://en.wikipedia.org/w/index.php?title=" + page_title + "&diff=prev&oldid=" + str(revid[1].reverting_id) + "\",\"diff\")"
                
                count = count + 1
                
            output_df = output_df.append({"reverting_bot":reverting_user_text_link,
                                          "reverted_bot":rev_user_text_link,
                                          "total_reverts": total_reverts,
                                          "ns0_reverts":ns0_reverts,
                                          "ex0":ex{0},
                                          "ex1":ex{1},
                                          "ex2":ex{2},
                                          "ex3":ex{3},
                                          "ex4":ex{4},
                                          "ex5":ex{5},
                                          "ex6":ex{6},
                                          "ex7":ex{7},
                                          "ex8":ex{8},
                                          "ex9":ex{9}},
                                         ignore_index=True)

SyntaxError: invalid syntax (<ipython-input-49-69398291b023>, line 30)

In [41]:
output_df

,reverting_bot,reverted_bot,total_reverts,ns0_reverts,examples
0,"<a href=""http://enwp.org/User:FrescoBot"">Fresc...","<a href=""http://enwp.org/User:Mathbot"">Mathbot...",58.0,58.0,"<a href=""https://en.wikipedia.org/w/index.php?..."
1,"<a href=""http://enwp.org/User:Addbot"">Addbot</a>","<a href=""http://enwp.org/User:Chobot"">Chobot</a>",1825.0,154.0,"<a href=""https://en.wikipedia.org/w/index.php?..."
2,"<a href=""http://enwp.org/User:Xqbot"">Xqbot</a>","<a href=""http://enwp.org/User:Chobot"">Chobot</a>",34.0,34.0,"<a href=""https://en.wikipedia.org/w/index.php?..."
3,"<a href=""http://enwp.org/User:EmausBot"">EmausB...","<a href=""http://enwp.org/User:Chobot"">Chobot</a>",294.0,92.0,"<a href=""https://en.wikipedia.org/w/index.php?..."
4,"<a href=""http://enwp.org/User:KLBot2"">KLBot2</a>","<a href=""http://enwp.org/User:Chobot"">Chobot</a>",172.0,7.0,"<a href=""https://en.wikipedia.org/w/index.php?..."
5,"<a href=""http://enwp.org/User:タチコマ robot"">タチコマ...","<a href=""http://enwp.org/User:RussBot"">RussBot...",2056.0,1981.0,"<a href=""https://en.wikipedia.org/w/index.php?..."
6,"<a href=""http://enwp.org/User:Cydebot"">Cydebot...","<a href=""http://enwp.org/User:RussBot"">RussBot...",964.0,882.0,"<a href=""https://en.wikipedia.org/w/index.php?..."
7,"<a href=""http://enwp.org/User:BOTijo"">BOTijo</a>","<a href=""http://enwp.org/User:RussBot"">RussBot...",31.0,31.0,"<a href=""https://en.wikipedia.org/w/index.php?..."
8,"<a href=""http://enwp.org/User:Addbot"">Addbot</a>","<a href=""http://enwp.org/User:RussBot"">RussBot...",402.0,396.0,"<a href=""https://en.wikipedia.org/w/index.php?..."
9,"<a href=""http://enwp.org/User:Xqbot"">Xqbot</a>","<a href=""http://enwp.org/User:RussBot"">RussBot...",5163.0,4689.0,"<a href=""https://en.wikipedia.org/w/index.php?..."


In [46]:
!pip install openpyxl
output_df.to_csv("revert_by_botpair.tsv", sep="\t")
output_df.to_excel("revert_by_botpair.xlsx")

    100% |████████████████████████████████| 184kB 382kB/s eta 0:00:01
  Using cached jdcal-1.3.tar.gz
  Using cached et_xmlfile-1.0.1.tar.gz
  Running setup.py install for jdcal ... - done
  Running setup.py install for et-xmlfile ... - done
  Running setup.py install for openpyxl ... - \ | / - done


In [43]:
!head revert_by_botpair.tsv

	reverting_bot	reverted_bot	total_reverts	ns0_reverts	examples
0	"<a href=""http://enwp.org/User:FrescoBot"">FrescoBot</a>"	"<a href=""http://enwp.org/User:Mathbot"">Mathbot</a>"	58.0	58.0	"<a href=""https://en.wikipedia.org/w/index.php?title=List_of_mathematicians_(Y)&diff=prev&oldid=506645953"">0</a> <a href=""https://en.wikipedia.org/w/index.php?title=List_of_mathematicians_(X)&diff=prev&oldid=518230338"">1</a> <a href=""https://en.wikipedia.org/w/index.php?title=List_of_mathematicians_(O)&diff=prev&oldid=487636640"">2</a> <a href=""https://en.wikipedia.org/w/index.php?title=List_of_mathematicians_(Z)&diff=prev&oldid=502248903"">3</a> <a href=""https://en.wikipedia.org/w/index.php?title=List_of_mathematicians_(O)&diff=prev&oldid=371875375"">4</a> <a href=""https://en.wikipedia.org/w/index.php?title=List_of_mathematicians_(J)&diff=prev&oldid=381630028"">5</a> <a href=""https://en.wikipedia.org/w/index.php?title=List_of_mathematicians_(X)&diff=prev&oldid=421732463"">6</a> <a href=""ht